# Extracting and cleaning 2025-2026 London LFR deployment data

- extract data from pdf
- inspect for issues
- convert street adresses to geocode coordinates 

In [1]:
%pip install tabula-py geopy pandas geopandas


   ---------------------------------------- 0.0/12.0 MB ? eta -:--:--
   ----- ---------------------------------- 1.6/12.0 MB 7.6 MB/s eta 0:00:02
   ------------- -------------------------- 3.9/12.0 MB 9.4 MB/s eta 0:00:01
   -------------------- ------------------- 6.0/12.0 MB 9.5 MB/s eta 0:00:01
   -------------------------- ------------- 7.9/12.0 MB 9.4 MB/s eta 0:00:01
   --------------------------------- ------ 10.0/12.0 MB 9.4 MB/s eta 0:00:01
   -------------------------------------- - 11.5/12.0 MB 9.2 MB/s eta 0:00:01
   ---------------------------------------- 12.0/12.0 MB 8.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/22.9 MB ? eta -:--:--
   -- ------------------------------------- 1.3/22.9 MB 7.4 MB/s eta 0:00:03
   ----- ---------------------------------- 3.1/22.9 MB 7.7 MB/s eta 0:00:03
   -------- ------------------------------- 5.0/22.9 MB 7.9 MB/s eta 0:00:03
   ----------- ---------------------------- 6.8/22.9 MB 8.2 MB/s eta 0:00:02
   ------

In [4]:
%pip install JPype1

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from pathlib import Path
import pandas as pd

#obtain clean file path
lfr_file_path = Path("C:/Users/barne/OneDrive/Documents/GitHub/london-lfr-surveillance/data/raw/live-facial-recognition---deployment-record-2025-to-date.pdf")


            Deployment Date  Unnamed: 0  Duration LFR Use Case Watchlist  \
0                  Location         NaN                    NaN      Size   
1                       NaN         NaN                    NaN       NaN   
2   High St North, 08/01/25         NaN  4hr 47m Crime Hotspot     15095   
3                  East Ham         NaN                    NaN       NaN   
4        Green St, 08/01/25         NaN  5hr 37m Crime Hotspot     15095   
..                      ...         ...                    ...       ...   
0                       NaN         NaN                    NaN       NaN   
1                       NaN         NaN                    NaN       NaN   
2                       NaN         NaN                    NaN       NaN   
3                       NaN         NaN                    NaN       NaN   
4                       NaN         NaN                    NaN       NaN   

          Min   Total True Alerts True Alerts.1 False Alerts False Alerts.1  \
0   Thre

In [12]:
%pip install pymupdf


   ---------------------------------------- 0.0/18.4 MB ? eta -:--:--
   ------ --------------------------------- 3.1/18.4 MB 18.5 MB/s eta 0:00:01
   -------------- ------------------------- 6.6/18.4 MB 19.2 MB/s eta 0:00:01
   --------------------- ------------------ 9.7/18.4 MB 18.3 MB/s eta 0:00:01
   --------------------------- ------------ 12.6/18.4 MB 15.5 MB/s eta 0:00:01
   ----------------------------------- ---- 16.5/18.4 MB 16.0 MB/s eta 0:00:01
   ---------------------------------------- 18.4/18.4 MB 15.1 MB/s eta 0:00:00


In [ ]:
import fitz
import pandas as pd

doc = fitz.open(lfr_file_path)
all_rows = []

for i, page in enumerate(doc):
    tabs = page.find_tables()
    
    if tabs.tables:
        raw_data = tabs[0].extract()
        
        if i == 0:
            all_rows.extend(raw_data[2:])
        else:
            all_rows.extend(raw_data[:])
    else:
        print(f"Page {i} is empty or has no table; skipping...")

df_here = pd.DataFrame(all_rows)

doc.close()

print(f"Total rows extracted: {len(df_here)}")

Page 10 is empty or has no table; skipping...
Total rows extracted: 465


In [82]:
# ok so now I need to add headers + delete first row + delete blank rows 

headers = [
'Deployment Location',
'Date',
'Duration',
'LFR Use Case',
'Watchlist Size',
'Min Threshold Setting',
'Total Alerts',
'True Alerts Confirmed',
'True Alerts Unconfirmed',
'False Alerts Confirmed',
'False Alerts Unconfirmed',
'False Alert Rate',
'Outcome - Arrest',
'Outcome - Other',
'No action',
'Faces seen (estimate)']

df_here.columns = headers

#df_here.drop(index=df_here.index[0], axis=0, inplace=True)

df_here = df_here.dropna(how='any')



In [83]:
df_here.reset_index(drop=True, inplace=True)
df_here['Deployment Location'] = df_here['Deployment Location'].str.replace("\n", " ")
print(df_here.head())

       Deployment Location      Date Duration   LFR Use Case Watchlist Size  \
0  High St North, East Ham  08/01/25  4hr 47m  Crime Hotspot          15095   
1         Green St, Newham  08/01/25  5hr 37m  Crime Hotspot          15095   
2       Powis St, Woolwich  10/01/25  6hr 27m  Crime Hotspot          15133   
3         Camberwell Green  10/01/25  5hr 58m  Crime Hotspot          15133   
4        Rye Lane, Peckham  14/01/25  6hr 20m  Crime Hotspot          15177   

  Min Threshold Setting Total Alerts True Alerts Confirmed  \
0                  0.64           10                    10   
1                  0.64            6                     6   
2                  0.64           14                    14   
3                  0.64            4                     4   
4                  0.64           12                    11   

  True Alerts Unconfirmed False Alerts Confirmed False Alerts Unconfirmed  \
0                       0                      0                        0  

In [73]:
%pip install geopy

Note: you may need to restart the kernel to use updated packages.


In [ ]:
#now to geocode 

geo_lfr_df = df_here.copy()

geo_lfr_df["Deployment Location"] = (
    geo_lfr_df["Deployment Location"]
    .str.replace("’", "'", regex=False)
    .str.replace("–", "-", regex=False)
    .str.replace(r"\bSt\b", "Street", regex=True)
    .str.replace(r"\bRd\b", "Road", regex=True)
    .str.replace(r"\bSq\b", "Square", regex=True)
    .str.replace(r"\bStn\b", "Station", regex=True)
    .str.replace("B'Way", "Broadway", regex=False)
    .str.replace("Junc", "Junction", regex=False)
    .str.strip()
)


In [74]:
location_names = geo_lfr_df['Deployment Location'].to_list()

In [91]:
from geopy.geocoders import Nominatim
import time

geolocator = Nominatim(user_agent="lfr_deployment")

latitudes = []
longitudes = []
errors = []

for i, address in enumerate(geo_lfr_df["Deployment Location"]):
    try:
        query = f"{address}, London, UK"
        location = geolocator.geocode(query)
    except Exception as e:
        print(f"Error at row {i}: {e}")
        errors.append(i)
        location = None

    if location:
        latitudes.append(location.latitude)
        longitudes.append(location.longitude)
    else:
        latitudes.append(None)
        longitudes.append(None)
        print(f"Row {i}: '{address}' could not be geocoded")

    time.sleep(1)

geo_lfr_df["latitude"] = latitudes
geo_lfr_df["longitude"] = longitudes

print("Completed")


Row 26: 'Westfield, Shepherds Bush' could not be geocoded
Row 39: 'Ealing B'way' could not be geocoded
Row 87: 'Tooting B'way' could not be geocoded
Row 94: 'Ealing B'way' could not be geocoded
Row 98: 'Poplar - Vesey Path' could not be geocoded
Error at row 102: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=Kings+Cross+Station%2C+London%2C+UK&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))
Row 102: 'Kings Cross Station' could not be geocoded
Row 103: 'Uxbridge Road, Shepherds Bush' could not be geocoded
Row 119: 'Piccadilly, Hard Rock' could not be geocoded
Row 130: 'Stratford, Westfields' could not be geocoded
Row 171: 'East Ham, High Road North' could not be geocoded
Row 198: 'Piccadilly, Hard Rock' could not be geocoded
Row 207: 'Westfield, White City' could not be geocoded
Row 227: 'Dalston Junctiontion' could not be 

In [ ]:
geo_lfr_df["Deployment Location"] = (
    geo_lfr_df["Deployment Location"]
    .str.replace("Junctiontion", "Junction", regex=False)
    .str.replace("B'way", "Broadway", regex=False)
)

In [ ]:
# manually assing the missing rows 
# https://www.gps-coordinates.net/

geo_lfr_df.loc[26, ["latitude", "longitude"]] = [51.506255, -0.220575]
geo_lfr_df.loc[39, ["latitude", "longitude"]] = [51.51498, -0.300407]
geo_lfr_df.loc[87, ["latitude", "longitude"]] = [51.427739, -0.16829]
geo_lfr_df.loc[94, ["latitude", "longitude"]] = [51.51498, -0.300407]
geo_lfr_df.loc[98, ["latitude", "longitude"]] = [51.511903, -0.014372]
geo_lfr_df.loc[102, ["latitude", "longitude"]] = [51.531707, -0.124766]
geo_lfr_df.loc[103, ["latitude", "longitude"]] = [51.506295, -0.231435]

#need to complete the rest

